In [1]:
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
data_cp = pd.read_excel("./data.xlsx")

In [3]:
data_cp.head()

,Unnamed: 0,Company_name,Company_area,Company_code,Company_industry,CITI,CITI_all_rank,CITI_trade_rank,CATI,CATI_all_rank,...,Band_18,Band_19,Band_20,Band_21,Band_22,Band_23,Band_24,Band_25,Band_26,Band_27
0,0,H&M,瑞典,-,"纺织, 皮革",49.22,28,9,53.8,30,...,green,green,green,green,green,green,green,green,grey,green
1,1,Gap,美国,GPS.N,纺织,62.90,20,15,56.0,26,...,green,green,green,green,grey,green,green,green,grey,green
2,2,C&A,德国,-,纺织,70.48,13,10,47.4,50,...,green,green,grey,grey,grey,green,grey,grey,grey,green
3,3,惠普,美国,HPQ.N,IT/ ICT,35.08,46,11,50.4,40,...,green,green,grey,green,grey,green,green,green,grey,green
4,4,微软,美国,MSFT.O,IT/ ICT,67.78,15,4,69.4,11,...,green,green,green,green,green,green,green,green,green,green


In [4]:
data_cn = data_cp.copy()
data_cn = data_cn.drop(['Unnamed: 0'], axis=1)
data_cn = data_cn[data_cn["Company_area"] == "中国大陆"]

In [29]:
data_cn_lc = data_cn.copy()
data_cn_lc = data_cn_lc[data_cn_lc["Company_code"] != "-"]

In [9]:
cpc = data_cn_lc.iloc[0].Company_code

In [10]:
url = "https://esg.api.ccxgf.com/common/esgCompanySearch?name="

In [11]:
response = requests.request("GET", url + cpc)

In [13]:
if response.status_code == 200:
    cp_info = response.json()
    if cp_info["msg"] == "请求成功":
        print("Success")
    else:
        print("Error")
else:
    print("Error")

Success


In [14]:
cp_info["result"]["data"][0]["securityCode"]

'002331.SZ'

In [15]:
cp_info["result"]["data"][1]["securityCode"]

'02331.HK'

In [18]:
cp_codes = [cp_info["result"]["data"][i]["securityCode"] for i in range(len(cp_info["result"]["data"]))]

In [20]:
min(zip(cp_codes, [len(cp_codes[i]) for i in range(len(cp_codes))]), key=lambda x: x[1])[0]

'02331.HK'

In [31]:
def get_cp_code(cp_code):
    url = "https://esg.api.ccxgf.com/common/esgCompanySearch?name="
    response = requests.request("GET", url + cp_code)
    if response.status_code == 200:
        cp_info = response.json()
        if cp_info["msg"] == "请求成功":
            if len(cp_info["result"]["data"]) == 0:
                return None
            cp_codes = [cp_info["result"]["data"][i]["securityCode"] for i in range(len(cp_info["result"]["data"]))]
            return min(zip(cp_codes, [len(cp_codes[i]) for i in range(len(cp_codes))]), key=lambda x: x[1])[0]
        else:
            return None
    else:
        return None

In [32]:
data_cn_lc["Company_code"] = data_cn_lc["Company_code"].apply(get_cp_code)

In [33]:
data_cn_lc.drop(data_cn_lc[data_cn_lc["Company_code"].isnull()].index, inplace=True)

In [36]:
data_cn_lc.to_excel("./data_cn_lc.xlsx", index=False)

In [37]:
data_cn_lc["Environment"] = None

In [38]:
data_cn_lc["Social"] = None
data_cn_lc["Governance"] = None
data_cn_lc["ESG_ccxgf"] = None
data_cn_lc["ESG_iigf"] = None

In [70]:
import requests

url = "https://stock-data.nbd.com.cn/api/1/esg/companies/info?code=300785.SZ"

payload = {}
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Encoding': 'gzip, deflate, br',
  'Host': 'stock-data.nbd.com.cn',
  'Origin': 'https://www.nbd.com.cn',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"result":1,"pin_pai_bang":"","company_rating_time_str":"2022","company_rating":{"time_str":"2022","rating_date":"2022-01-01","code":"300785.SZ","market":"SZ","company_name":"值得买","industry_lev2":"互联网和相关服务","esg_lev":"A+","esg_index":1,"esg_index_s":"1/69","esg_index_percent":1.44,"esg_index_percent_above":98.56},"lev_sumary":{"result":1,"lev_each":[{"esg_lev":"D-","count":0},{"esg_lev":"D","count":5},{"esg_lev":"D+","count":9},{"esg_lev":"C-","count":6},{"esg_lev":"C","count":13},{"esg_lev":"C+","count":6},{"esg_lev":"B-","count":8},{"esg_lev":"B","count":7},{"esg_lev":"B+","count":5},{"esg_lev":"A-","count":6},{"esg_lev":"A","count":1},{"esg_lev":"A+","count":3}],"levs_sum":[["D",14],["C",25],["B",20],["A",10]]},"tanpaifang_bangdans_time_str":"","tanpaifang_bangdans":{}}


In [73]:
data_esg = response.json()
data_esg["company_rating"]["esg_lev"]

'A+'

In [80]:
def get_esg_iigf(cp_code):
    url = "https://stock-data.nbd.com.cn/api/1/esg/companies/info?code=" + cp_code

    payload = {}
    headers = {
      'Accept': 'application/json, text/plain, */*',
      'Accept-Encoding': 'gzip, deflate, br',
      'Host': 'stock-data.nbd.com.cn',
      'Origin': 'https://www.nbd.com.cn',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'
    }
    
    response = requests.request("GET", url, headers=headers, data=payload)
    data_esg = response.json()
    if response.status_code == 200:
        if data_esg["company_rating"] != "":
            esg = data_esg["company_rating"]["esg_lev"]
            return esg
        return None
    return None

In [48]:
if response.status_code == 200:
    if data_esg["msg"] == "请求成功":
        environment = data_esg["result"]["data"]["environment"]
        social = data_esg["result"]["data"]["social"]
        governance = data_esg["result"]["data"]["governance"]
        ratingLevel = data_esg["result"]["data"]["ratingLevel"]

In [51]:
def get_esg_ccxgf(cp_code):
    url = "https://esg.api.ccxgf.com/common/esgRatingSearch?name=" + cp_code

    payload = {}
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0',
      'Sec-Fetch-Site': 'cross-site',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Dest': 'empty',
      'Sec-Ch-Ua-Platform': '"Windows"',
      'Sec-Ch-Ua-Mobile': '?0',
      'Sec-Ch-Ua': '"Microsoft Edge";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
      'Referer': 'http://esgratings.ccxgf.com/',
      'Origin': 'http://esgratings.ccxgf.com',
      'Host': 'esg.api.ccxgf.com'
    }
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    data_esg = response.json()
    if response.status_code == 200:
        if data_esg["msg"] == "请求成功":
            environment = data_esg["result"]["data"]["environment"]
            social = data_esg["result"]["data"]["social"]
            governance = data_esg["result"]["data"]["governance"]
            ratingLevel = data_esg["result"]["data"]["ratingLevel"]
            return [environment, social, governance, ratingLevel]
        else:
            return None
    else:
        return None
    

In [56]:
environment = []
social = []
governance = []
esg_ccxgf = []
for i in tqdm(range(0, len(data_cn_lc))):
    data_esg = get_esg_ccxgf(data_cn_lc.iloc[i].Company_code)
    if data_esg != None:
        environment.append(data_esg[0]) 
        social.append(data_esg[1])
        governance.append(data_esg[2])
        esg_ccxgf.append(data_esg[3])

100%|██████████| 215/215 [01:28<00:00,  2.43it/s]


In [57]:
data_cn_lc["Environment"] = environment

In [58]:
data_cn_lc["Social"] = social

In [59]:
data_cn_lc["Governance"] = governance

In [60]:
data_cn_lc["ESG_ccxgf"] = esg_ccxgf

In [61]:
data_cn_lc.head()

,Company_name,Company_area,Company_code,Company_industry,CITI,CITI_all_rank,CITI_trade_rank,CATI,CATI_all_rank,CATI_trade_rank,...,Band_23,Band_24,Band_25,Band_26,Band_27,Environment,Social,Governance,ESG_ccxgf,ESG_iigf
22,李宁有限公司,中国大陆,02331.HK,"纺织, 皮革",39.46,39,11,24.8,284,32,...,green,green,green,grey,grey,B,A⁻,BB,BB,None
26,联想集团有限公司,中国大陆,00992.HK,IT/ ICT,46.44,34,9,65.2,15,7,...,green,green,green,green,green,BB,A,BB,BB,None
29,雅戈尔集团股份有限公司,中国大陆,600177.SH,纺织,6.40,531,108,7.0,929,117,...,grey,green,green,grey,green,BB,A⁺,A⁺,A⁻,None
44,探路者控股集团股份有限公司,中国大陆,300005.SZ,"纺织, 皮革",2.00,647,59,2.5,1269,59,...,grey,grey,grey,grey,grey,C,BB,A⁻,BB,None
51,青岛啤酒股份有限公司,中国大陆,"600600.SH,00168.HK",啤酒,10.70,358,6,16.0,508,6,...,green,green,green,green,green,BBB,AA⁺,AA,AA⁻,None


In [81]:
esg_iigf = []
for i in tqdm(range(0, len(data_cn_lc))):
    esg = get_esg_iigf(data_cn_lc.iloc[i].Company_code)
    esg_iigf.append(esg)

100%|██████████| 215/215 [01:10<00:00,  3.06it/s]


In [83]:
data_cn_lc["ESG_iigf"] = esg_iigf

In [85]:
data_cn_lc

,Company_name,Company_area,Company_code,Company_industry,CITI,CITI_all_rank,CITI_trade_rank,CATI,CATI_all_rank,CATI_trade_rank,...,Band_23,Band_24,Band_25,Band_26,Band_27,Environment,Social,Governance,ESG_ccxgf,ESG_iigf
22,李宁有限公司,中国大陆,02331.HK,"纺织, 皮革",39.46,39,11,24.8,284,32,...,green,green,green,grey,grey,B,A⁻,BB,BB,None
26,联想集团有限公司,中国大陆,00992.HK,IT/ ICT,46.44,34,9,65.2,15,7,...,green,green,green,green,green,BB,A,BB,BB,None
29,雅戈尔集团股份有限公司,中国大陆,600177.SH,纺织,6.40,531,108,7.0,929,117,...,grey,green,green,grey,green,BB,A⁺,A⁺,A⁻,B+
44,探路者控股集团股份有限公司,中国大陆,300005.SZ,"纺织, 皮革",2.00,647,59,2.5,1269,59,...,grey,grey,grey,grey,grey,C,BB,A⁻,BB,C+
51,青岛啤酒股份有限公司,中国大陆,"600600.SH,00168.HK",啤酒,10.70,358,6,16.0,508,6,...,green,green,green,green,green,BBB,AA⁺,AA,AA⁻,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,周黑鸭国际控股有限公司,中国大陆,01458.HK,食品饮料,7.54,487,48,10.2,753,44,...,green,green,green,green,grey,BB,A,BB,BBB⁻,None
695,嘉士利集团有限公司,中国大陆,01285.HK,食品饮料,8.92,433,44,9.6,782,48,...,grey,green,green,grey,grey,BB,A⁺,BB,BBB⁻,None
698,绿城中国控股有限公司,中国大陆,03900.HK,房地产建筑,8.08,463,42,16.4,496,9,...,green,green,green,green,grey,B,BBB⁺,BB,BB,None
700,深圳传音控股股份有限公司,中国大陆,688036.SH,IT/ ICT,9.00,428,60,5.0,1054,94,...,grey,grey,green,grey,grey,A,AAA,AA⁻,AA⁻,A


In [86]:
data_cn_lc.to_excel("data_cn_lc.xlsx")

In [92]:
data_hz = pd.read_stata("./macrodata_basic.dta")

In [93]:
max(data_hz.年份)

2021.0

In [99]:
import requests

url = "https://global.finance.sina.com.cn/api/openapi.php/EsgService.getEsgStocks?page="

payload = {}
headers = {}
data_esg_all = []
for i in range(1, 31):
    response = requests.request("GET", url + str(i), headers=headers, data=payload)
    data = response.json()
    data = data["result"]["data"]['info']["stocks"]
    data_esg_all.extend(data)


In [101]:
data_esg_all

[{'symbol': 'SZ000001',
  'market': 'cn',
  'esg_info': [{'agency': 'agency2',
    'agency_name': '中财绿金院',
    'esg_score': 'A-',
    'esg_dt': '2022Q4'},
   {'agency': 'agency1',
    'agency_name': '商道融绿',
    'esg_score': 'A-',
    'esg_dt': '2023Q3'},
   {'agency': 'agency4',
    'agency_name': '盟浪',
    'esg_score': 'A+',
    'esg_dt': '2022Q4'},
   {'agency': 'agency7',
    'agency_name': '中诚信',
    'esg_score': 'AA-',
    'esg_dt': '2023Q4'},
   {'agency': 'agency11',
    'agency_name': '晨星Sustainalytics',
    'esg_score': '-',
    'esg_dt': '2023Q3',
    'remark': ''},
   {'agency': 'agency9',
    'agency_name': '妙盈',
    'esg_score': '-',
    'esg_dt': '2023Q3'},
   {'agency': 'agency5',
    'agency_name': '华测CTI',
    'esg_score': 'CCC',
    'esg_dt': '2022Q1'}]},
 {'symbol': 'SZ000002',
  'market': 'cn',
  'esg_info': [{'agency': 'agency2',
    'agency_name': '中财绿金院',
    'esg_score': 'A',
    'esg_dt': '2022Q4'},
   {'agency': 'agency1',
    'agency_name': '商道融绿',
    'esg_s

In [2]:
import json

In [3]:
with open("esg_data.json", "r") as f:
    data_esg_all = json.load(f)

In [5]:
cp_1  = data_esg_all[0]

In [7]:
cp_1["symbol"]

'SZ000001'

In [8]:
cp_1["market"]

'cn'

In [25]:
cp_1["esg_info"]

[{'agency': 'agency2',
  'agency_name': '中财绿金院',
  'esg_score': 'A-',
  'esg_dt': '2022Q4'},
 {'agency': 'agency1',
  'agency_name': '商道融绿',
  'esg_score': 'A-',
  'esg_dt': '2023Q3'},
 {'agency': 'agency4',
  'agency_name': '盟浪',
  'esg_score': 'A+',
  'esg_dt': '2022Q4'},
 {'agency': 'agency7',
  'agency_name': '中诚信',
  'esg_score': 'AA-',
  'esg_dt': '2023Q4'},
 {'agency': 'agency11',
  'agency_name': '晨星Sustainalytics',
  'esg_score': '-',
  'esg_dt': '2023Q3',
  'remark': ''},
 {'agency': 'agency9',
  'agency_name': '妙盈',
  'esg_score': '-',
  'esg_dt': '2023Q3'},
 {'agency': 'agency5',
  'agency_name': '华测CTI',
  'esg_score': 'CCC',
  'esg_dt': '2022Q1'}]

In [24]:
cp_name = []
ag_1 = [] # 中财绿金院
ag_1_dt = []
ag_2 = [] # 商道融绿
ag_2_dt = []
ag_3 = [] # 盟浪
ag_3_dt = []
ag_4 = [] # 中诚信
ag_4_dt = []
ag_5 = [] # 晨星Sustainalytics
ag_5_dt = []
ag_6 = [] # 妙盈
ag_6_dt = []
ag_7 = [] # 华测CTI
ag_7_dt = []

In [29]:
for i in range(len(data_esg_all)):
    cp_name.append(data_esg_all[i]["symbol"])
    ag_1.append(data_esg_all[i]["esg_info"][0]["esg_score"])
    ag_1_dt.append(data_esg_all[i]["esg_info"][0]["esg_dt"])
    ag_2.append(data_esg_all[i]["esg_info"][1]["esg_score"])
    ag_2_dt.append(data_esg_all[i]["esg_info"][1]["esg_dt"])
    ag_3.append(data_esg_all[i]["esg_info"][2]["esg_score"])
    ag_3_dt.append(data_esg_all[i]["esg_info"][2]["esg_dt"])
    ag_4.append(data_esg_all[i]["esg_info"][3]["esg_score"])
    ag_4_dt.append(data_esg_all[i]["esg_info"][3]["esg_dt"])
    ag_5.append(data_esg_all[i]["esg_info"][4]["esg_score"])
    ag_5_dt.append(data_esg_all[i]["esg_info"][4]["esg_dt"])
    ag_6.append(data_esg_all[i]["esg_info"][5]["esg_score"])
    ag_6_dt.append(data_esg_all[i]["esg_info"][5]["esg_dt"])
    ag_7.append(data_esg_all[i]["esg_info"][6]["esg_score"])
    ag_7_dt.append(data_esg_all[i]["esg_info"][6]["esg_dt"])


In [30]:
data_esg = pd.DataFrame({"Company_name": cp_name, "中财绿金院": ag_1, "中财绿金院_dt": ag_1_dt, "商道融绿": ag_2, "商道融绿_dt": ag_2_dt, "盟浪": ag_3, "盟浪_dt": ag_3_dt, "中诚信": ag_4, "中诚信_dt": ag_4_dt, "晨星Sustainalytics": ag_5, "晨星Sustainalytics_dt": ag_5_dt, "妙盈": ag_6, "妙盈_dt": ag_6_dt, "华测CTI": ag_7, "华测CTI_dt": ag_7_dt})


In [31]:
data_esg.to_excel("./data_esg.xlsx", index=False)

In [32]:
def change_cp_name(cp_name):
    return cp_name[2:]+"."+cp_name[:2]

In [34]:
change_cp_name(cp_name[0])

'000001.SZ'

In [35]:
data_esg["Company_code"] = data_esg["Company_name"].apply(change_cp_name)

In [37]:
data_cn_lc = pd.read_excel("./data_cn_lc.xlsx")

In [38]:
data_cn_lc.head()

,Unnamed: 0,Company_name,Company_area,Company_code,Company_industry,CITI,CITI_all_rank,CITI_trade_rank,CATI,CATI_all_rank,...,Band_23,Band_24,Band_25,Band_26,Band_27,Environment,Social,Governance,ESG_ccxgf,ESG_iigf
0,22,李宁有限公司,中国大陆,02331.HK,"纺织, 皮革",39.46,39,11,24.8,284,...,green,green,green,grey,grey,B,A⁻,BB,BB,NaN
1,26,联想集团有限公司,中国大陆,00992.HK,IT/ ICT,46.44,34,9,65.2,15,...,green,green,green,green,green,BB,A,BB,BB,NaN
2,29,雅戈尔集团股份有限公司,中国大陆,600177.SH,纺织,6.40,531,108,7.0,929,...,grey,green,green,grey,green,BB,A⁺,A⁺,A⁻,B+
3,44,探路者控股集团股份有限公司,中国大陆,300005.SZ,"纺织, 皮革",2.00,647,59,2.5,1269,...,grey,grey,grey,grey,grey,C,BB,A⁻,BB,C+
4,51,青岛啤酒股份有限公司,中国大陆,"600600.SH,00168.HK",啤酒,10.70,358,6,16.0,508,...,green,green,green,green,green,BBB,AA⁺,AA,AA⁻,NaN


In [39]:
ESG_iigf_sf = []
ESG_syntaogf = []
ESG_susallwave = []
ESG_ccxi = []
ESG_sustainalytics = []
ESG_miotech = []
ESG_cti = []

In [42]:
for i in range(len(data_cn_lc)):
    cp_code = data_cn_lc.iloc[i].Company_code
    if cp_code in data_esg["Company_code"].values:
        ESG_iigf_sf.append(data_esg[data_esg["Company_code"] == cp_code]["中财绿金院"].values[0])
        ESG_syntaogf.append(data_esg[data_esg["Company_code"] == cp_code]["商道融绿"].values[0])
        ESG_susallwave.append(data_esg[data_esg["Company_code"] == cp_code]["盟浪"].values[0])
        ESG_ccxi.append(data_esg[data_esg["Company_code"] == cp_code]["中诚信"].values[0])
        ESG_sustainalytics.append(data_esg[data_esg["Company_code"] == cp_code]["晨星Sustainalytics"].values[0])
        ESG_miotech.append(data_esg[data_esg["Company_code"] == cp_code]["妙盈"].values[0])
        ESG_cti.append(data_esg[data_esg["Company_code"] == cp_code]["华测CTI"].values[0])
    else:
        ESG_iigf_sf.append(None)
        ESG_syntaogf.append(None)
        ESG_susallwave.append(None)
        ESG_ccxi.append(None)
        ESG_sustainalytics.append(None)
        ESG_miotech.append(None)
        ESG_cti.append(None)

In [50]:
for i in range(len(ESG_iigf_sf)):
    if ESG_iigf_sf[i] == "-":
        ESG_iigf_sf[i] = None
    if ESG_syntaogf[i] == "-":
        ESG_syntaogf[i] = None
    if ESG_susallwave[i] == "-":
        ESG_susallwave[i] = None
    if ESG_ccxi[i] == "-":
        ESG_ccxi[i] = None
    if ESG_sustainalytics[i] == "-":
        ESG_sustainalytics[i] = None
    if ESG_miotech[i] == "-":
        ESG_miotech[i] = None
    if ESG_cti[i] == "-":
        ESG_cti[i] = None


In [51]:
data_cn_lc["ESG_iigf_sf"] = ESG_iigf_sf
data_cn_lc["ESG_syntaogf"] = ESG_syntaogf
data_cn_lc["ESG_susallwave"] = ESG_susallwave
data_cn_lc["ESG_ccxi"] = ESG_ccxi
data_cn_lc["ESG_sustainalytics"] = ESG_sustainalytics
data_cn_lc["ESG_miotech"] = ESG_miotech
data_cn_lc["ESG_cti"] = ESG_cti


In [53]:
data_cn_lc.to_excel("./data_cn_lc.xlsx", index=False)